In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Read the CSV file
df = pd.read_csv('experiments/LSTM_phase1_exp02_20251105_000850/metrics/comparison_fold_35_validation.csv')

# Convert target_time to datetime with UTC timezone
df['target_time'] = pd.to_datetime(df['target_time'], utc=True)

# Convert to Pacific Time (handling DST automatically)
df['target_time_pt'] = df['target_time'].dt.tz_convert('US/Pacific')
df['target_time_pt_str'] = df['target_time_pt'].dt.strftime('%Y-%m-%d %H:%M:%S %Z')

# Handle missing values in NAM columns
df['nam_ghi'] = pd.to_numeric(df['nam_ghi'], errors='coerce')
df['nam_csi'] = pd.to_numeric(df['nam_csi'], errors='coerce')

# Create custom hover data
hover_data = np.column_stack((
    df['fold_id'].astype(str),
    df['sample_index'].astype(str),
    df['sample_order'].astype(str),
    df['bin_id'].astype(str),
    df['target_date'].astype(str)
))

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# GHI features (primary y-axis)
ghi_features = ['clear_sky_ghi', 'actual_ghi', 'nam_ghi', 'model_ghi_pred']
ghi_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']  # Distinct colors
ghi_names = ['Clear Sky GHI', 'Actual GHI', 'NAM GHI', 'Model GHI Pred']

for i, feature in enumerate(ghi_features):
    fig.add_trace(
        go.Scatter(
            x=df['target_time'],
            y=df[feature],
            name=ghi_names[i],
            mode='lines+markers',
            line=dict(width=2, color=ghi_colors[i]),
            marker=dict(size=6),
            customdata=hover_data,
            text=df['target_time_pt_str'],
            hovertemplate=(
                "<b>%{fullData.name}</b><br>"
                "Value: %{y:.4f} W/m²<br>"
                "UTC Time: %{x|%Y-%m-%d %H:%M:%S}<br>"
                "PT Time: %{text}<br>"
                "Fold ID: %{customdata[0]}<br>"
                "Sample Index: %{customdata[1]}<br>"
                "Sample Order: %{customdata[2]}<br>"
                "Bin ID: %{customdata[3]}<br>"
                "Target Date: %{customdata[4]}<extra></extra>"
            )
        ),
        secondary_y=False
    )

# CSI features (secondary y-axis)
csi_features = ['actual_csi', 'nam_csi', 'model_csi_pred']
csi_colors = ['#9467bd', '#8c564b', '#e377c2']  # Distinct colors
csi_names = ['Actual CSI', 'NAM CSI', 'Model CSI Pred']

for i, feature in enumerate(csi_features):
    fig.add_trace(
        go.Scatter(
            x=df['target_time'],
            y=df[feature],
            name=csi_names[i],
            mode='lines+markers',
            line=dict(width=2, dash='dash', color=csi_colors[i]),
            marker=dict(size=6, symbol='diamond'),
            customdata=hover_data,
            text=df['target_time_pt_str'],
            hovertemplate=(
                "<b>%{fullData.name}</b><br>"
                "Value: %{y:.4f}<br>"
                "UTC Time: %{x|%Y-%m-%d %H:%M:%S}<br>"
                "PT Time: %{text}<br>"
                "Fold ID: %{customdata[0]}<br>"
                "Sample Index: %{customdata[1]}<br>"
                "Sample Order: %{customdata[2]}<br>"
                "Bin ID: %{customdata[3]}<br>"
                "Target Date: %{customdata[4]}<extra></extra>"
            )
        ),
        secondary_y=True
    )

# Get fold ID for title
fold_id = df['fold_id'].iloc[0]

# Update layout
fig.update_layout(
    title={
        'text': f"Solar Irradiance and Clear Sky Index (Dec 2-11, 2016) - Fold {fold_id}",
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 20}
    },
    xaxis_title="UTC Time",
    hovermode="x unified",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    template="plotly_white",
    height=700,
    margin=dict(t=80, b=60, l=60, r=60),
    hoverlabel=dict(bgcolor="white", font_size=12)
)

# Update y-axes titles and styling
fig.update_yaxes(
    title_text="GHI (W/m²)", 
    secondary_y=False,
    gridcolor='#eaeaea',
    title_font=dict(size=14, color='black'),
    tickfont=dict(color='black')
)

fig.update_yaxes(
    title_text="Clear Sky Index (CSI)", 
    secondary_y=True,
    range=[-0.1, 1.3],  # Slightly beyond min/max CSI values
    title_font=dict(size=14, color='black'),
    tickfont=dict(color='black')
)

# Add vertical lines separating days
unique_dates = sorted(df['target_date'].unique())
for date in unique_dates[1:]:
    fig.add_vline(
        x=pd.Timestamp(date), 
        line_dash="dot", 
        line_color="gray",
        opacity=0.5,
        layer="below"
    )

# Add annotations for sample transitions
for date in unique_dates:
    # Get sample info for this date
    sample_info = df[df['target_date'] == date].iloc[0]
    fig.add_annotation(
        x=pd.Timestamp(f"{date} 12:00:00"),  # Noon UTC for visibility
        y=0.95,
        yref="paper",
        text=f"Sample {sample_info['sample_index']}<br>(Order {sample_info['sample_order']})",
        showarrow=False,
        font=dict(size=12, color="black"),
        align="center",
        bgcolor="rgba(255, 255, 255, 0.85)",
        bordercolor="#444",
        borderwidth=1,
        borderpad=4
    )

# Add footnote about timezones
fig.add_annotation(
    text="All times shown in UTC. Hover for Pacific Time equivalents.",
    xref="paper", yref="paper",
    x=0.5, y=-0.12,
    showarrow=False,
    font=dict(size=11, color="gray"),
    align="center"
)

# Save the figure as an interactive HTML file instead of showing inline
output_filename = "solar_irradiance_fold35.html"
fig.write_html(output_filename)
print(f"Interactive plot saved successfully to: {output_filename}")
print("Open this file in any web browser to view the interactive visualization.")